In [2]:
import pydantic
print(pydantic.__version__)

2.12.3


In [18]:
from pydantic import BaseModel
from typing import Optional

class User(BaseModel):
    name: str
    age: int
    is_active: bool

user1 = User(name="Bob", age=30, is_active=True)
print(user1.model_dump())

{'name': 'Bob', 'age': 30, 'is_active': True}


In [19]:
class Student(BaseModel):
    name: str
    age: int = 18
    subject: str

student1 = Student(name="Jeff", age=16, subject="mathematics")
print(student1.model_dump())

{'name': 'Jeff', 'age': 16, 'subject': 'mathematics'}


In [5]:
user2 = User(name="Bob", age="35", is_active=True)
print(user2)
print(type(user2.age))

name='Bob' age=35 is_active=True
<class 'int'>


In [6]:
user2 = User(name="Bob", age="thirty", is_active=True)
print(user2)
print(type(user2.age))

ValidationError: 1 validation error for User
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='thirty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing

## Fields

In Pydantic, Field() helps to add metadata, validation rules, custom behavior to a model field

In [7]:
from pydantic import BaseModel, Field

class FieldUser(BaseModel):
    name: str = Field(description="The user's full name")

field_user1 = FieldUser(name="Vaibhav")
print(field_user1)

name='Vaibhav'


In [8]:
from pydantic import BaseModel, Field, ValidationError

class ProperUser(BaseModel):
    age: int = Field(default="Twelve", validate_default=True) #Int 12 or string 12 works for validation but not Twelve

proper_user1 = ProperUser()
print(proper_user1)

ValidationError: 1 validation error for ProperUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='Twelve', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing

## Aliasing 

Models use Aliasing to

1. Accept input (validation)
2. Give output (serialization)

Below is the scenairo where Aliasing can be helpful

Scenario:

Lets say data coming from JSON API has the following fields
1. pkg_weight_kg
2. pkg_dest
3. pkg_is_fragile

But there's a different requirement to use in class as follows 
1. weight
2. destination
3. is_fragile

In [9]:
class Package(BaseModel):
    weight: float = Field(alias="pkg_weight_kg")
    destination: str = Field(alias = "pkg_dest")
    is_fragile: bool = Field(alias = "pkg_is_fragile")


data = {
    "pkg_weight_kg" : 4.5,
    "pkg_is_fragile": True,
    "pkg_dest": "Singapore"
}

package = Package(**data)
print(package.weight)
print(package.destination)
print(package.is_fragile)

print(package.model_dump(by_alias=True))

4.5
Singapore
True
{'pkg_weight_kg': 4.5, 'pkg_dest': 'Singapore', 'pkg_is_fragile': True}


Theres a slight difference between validation alias and serialization alias. 

The string defined in validation alias accepts that field as input from the data coming 
The string defined in serialization alias outpts the field present while printing. When by_alias is used in print statement, it gives the serialization alias as the fieldß

In [10]:
class Student(BaseModel):
    email: str = Field(validation_alias="student_email", serialization_alias="studentEmail")


incoming_data = {"student_email": "hi@gmail.com"}

student = Student(**incoming_data)
print(student.email)

print(student.model_dump())
print(student.model_dump(by_alias=True))


hi@gmail.com
{'email': 'hi@gmail.com'}
{'studentEmail': 'hi@gmail.com'}


### Numerical Limits as part of validation

In [14]:
class Product(BaseModel):
    name: str = Field(min_length=1, max_length=25)
    price: float = Field(gt=0)
    description: str = Field(default=None, max_length=300)

valid_product = Product(name="Laptop", price="999", description="Laptop Cost")
print(valid_product)

name='Laptop' price=999.0 description='Laptop Cost'


In [16]:
valid_product = Product(name="Laptop", price="-999", description="Laptop Cost")


ValidationError: 1 validation error for Product
price
  Input should be greater than 0 [type=greater_than, input_value='-999', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than

Exercise 

Create a Book Model which has 

1. Required Title (string, 1-100 chars)
2. Required Author (string)
3. Optional ISBN
4. Price (positive float <= 1000)
5. in_stock(boolean default True)


In [27]:
class BookModel(BaseModel):
    title: str = Field(min_length=1, max_length=100)
    author : str
    isbn: Optional[str] = None
    price : float = Field(le=1000, gt=0)
    in_stock : bool = True

valid_book = BookModel(title="the friendship factor", author="XYZ", isbn="None", price="25.5", in_stock = True)
print(valid_book)

title='the friendship factor' author='XYZ' isbn='None' price=25.5 in_stock=True


### Model Validator 

1. @model_validator is a decorator and used to validate multiple fields together
2. It performs logic that involves the whole model
3. Run code before or after normal field validation

1. Before mode is before validation and before creating the object of the class. Here we can implement a class method to handle the error logic.
2. After validation happens after object of class is created. 

In [ ]:
from pydantic import model_validator

class Event(BaseModel):
    name: str
    start_hour: int
    end_hour: int

    @model_validator(mode="after")
    def check_time(self):
        if self.end_hour <= self.start_hour:
            raise ValueError("end_hour must be greater than start_hour. Please fix this")
        return self
    
event1 = Event(name="Hackathon", start_hour=10, end_hour=19)
# event1 = Event(name="Hackathon", start_hour=10, end_hour=9) # This fails because end_hour is less than start_hour


### Field Validator 


Unlike model which validates all variables, use field validator to validate one particular field

In [37]:
from pydantic import field_validator

class Product(BaseModel):
    price: float 

    @field_validator("price")
    def must_be_positive(value):
        if value < 0:
            raise ValueError("Price must be greater than 0")
        return value
    
product1 = Product(price=25)

### Built in Type Validators 

Pydantic support certain patterns no regex, no extra code

In [44]:
from pydantic import BaseModel, EmailStr, HttpUrl, PositiveInt

class Contact(BaseModel):
    email: EmailStr
    website: HttpUrl
    followers: PositiveInt

good = Contact(email = "example@example.com", website="https://www.example.com", followers=14)
print(good.model_dump())

{'email': 'example@example.com', 'website': HttpUrl('https://www.example.com/'), 'followers': 14}


### Nested Models 

Nested models are where one pydantic model lies inside of other model. Helps to organize the data easily 

In [49]:
class Address(BaseModel):
    street: str
    city: str 
    pincode: str

class User(BaseModel):
    name: str 
    email: EmailStr
    address: Address
    
data = {"name": "Robert", "email":"email@email.com", "address" : {"street": "1234-3", "city": "xyz", "pincode": "abc xyz"}}
user = User(**data)
print(user.address.city)

xyz


### Recursive Models 

Slightly different from Nested Models. It refers back to itself

In [59]:
from pydantic import BaseModel
from typing import List, Optional 

class FamilyTree(BaseModel):
    name: str
    children: Optional[List["FamilyTree"]] = None


FamilyTree.model_rebuild()

data = {
    "name":"root",
    "children": [{"name":"child1", "children": [{"name":"grandchild1"}, {"name":"grandchild2"}]}, {"name":"child2", "children": [{"name": "grandchild3"}]}]
}

family_tree = FamilyTree(**data)

In [60]:
print(family_tree.model_dump())

{'name': 'root', 'children': [{'name': 'child1', 'children': [{'name': 'grandchild1', 'children': None}, {'name': 'grandchild2', 'children': None}]}, {'name': 'child2', 'children': [{'name': 'grandchild3', 'children': None}]}]}


In [61]:
def print_family_tree(node, indent=0):
    print(" " * indent + node.name)
    if node.children:
        for child in node.children:
            print_family_tree(child, indent+2)

print_family_tree(family_tree)


root
  child1
    grandchild1
    grandchild2
  child2
    grandchild3
